**Задание для выполнения в классе**: прочитайте файл cta0001.wav с помощью `scipy.io.wavfile` и нарисуйте на графике спектр Фурье фрагмента сигнала, который начинается с 200 мс и длится 30 мс. Используйте любую оконную функцию.

In [ ]:
!wget https://pkholyavin.github.io/mastersprogramming/cta0001.wav

In [10]:
from scipy.fft import fft, fftfreq
from scipy.io import wavfile
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

In [47]:
fs, data = wavfile.read("cta0001.wav")
start_t = 0.2
frame_dur_t = 0.03
start_s = int(start_t * fs)
frame_dur_s = int(frame_dur_t * fs)

y = data[start_s:start_s + frame_dur_s]
x = np.linspace(0.0, frame_dur_s / fs, frame_dur_s, endpoint=False)

In [ ]:
plt.plot(x, y)
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
hann_window = signal.get_window("hann", frame_dur_s)
y_windowed = y * hann_window
plt.plot(x, y_windowed)
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
xf = fftfreq(frame_dur_s, 1 / fs)[:frame_dur_s//2]
yf_windowed = fft(y_windowed)
plt.plot(xf, 2.0/frame_dur_s * np.abs(yf_windowed[0:frame_dur_s//2]))
plt.xlabel("Frequency, Hz")
plt.ylabel("Amplitude")
plt.grid()
plt.show()

**Задание для выполнения в классе**: получите PitchTier для файла cta0001.wav с помощью библиотеки `parselmouth`. Используйте команду "Stylize", чтобы стилизовать контур с точностью 2 полутона, а затем интерполируйте стилизованный контур с помощью кубических сплайнов. 

In [ ]:
!pip install praat-parselmouth

In [13]:
import parselmouth

In [40]:
sound = parselmouth.Sound("cta0001.wav")
step, min_f0, max_f0 = 0.01, 75, 350  # s, Hz, Hz
manip = parselmouth.praat.call(sound, "To Manipulation", step, min_f0, max_f0)
pitch_tier = parselmouth.praat.call(manip, "Extract pitch tier")

In [41]:
parselmouth.praat.call(pitch_tier, "Stylize", 5, "Hz")

In [42]:
num_points = parselmouth.praat.call(pitch_tier, "Get number of points")
time_values = []
f0_values = []
for i in range(1, num_points + 1):
    time_values.append(parselmouth.praat.call(pitch_tier, "Get time from index", i))
    f0_values.append(parselmouth.praat.call(pitch_tier, "Get value at index", i))

In [43]:
from scipy.interpolate import CubicSpline
spl = CubicSpline(time_values, f0_values)

In [ ]:
xnew = np.linspace(0, max(time_values), num=1001)
plt.plot(xnew, spl(xnew), '-', label='cubic spline interpolation')
plt.plot(time_values, f0_values, 'o', label='stylized PitchTier')
plt.legend(loc='best')
plt.show()

**Задание для выполнения в классе**: обработайте сигнал сta0001.wav окнами по 30 мс с шагом 10 мс, на каждом окне вычислите максимум амплитуды. Постройте график.

In [48]:
import librosa
frame_length = int(0.03 * fs)
hop_length = int(0.01 * fs)
frames = librosa.util.frame(data, frame_length=frame_length, hop_length=hop_length, axis=0)

In [ ]:
max_ampl = [max(fr) for fr in frames]
plt.plot(max_ampl)
plt.xlabel("Frames")
plt.ylabel("Max amplitude")
plt.show()